In [0]:
import dlt
from pyspark.sql.functions import col, hash, current_timestamp, lit
from pyspark.sql import functions as F

In [0]:
param_environment = spark.conf.get("param_environment", "dev")
param_source_name = spark.conf.get("param_source_name", "")
schema = spark.conf.get("schema", "gold_schema")
param_scd_type = spark.conf.get("param_scd_type", "scd1")
silver_schema = spark.conf.get("silver_schema")

In [0]:
schema = spark.conf.get("gold_schema")

@dlt.table(
    name=f"gold.{schema}.all_books"
)
def get_all_books():
    df = dlt.readStream(f"{silver_schema}.{param_source_name}")
    df_all_books = (df
        .select(col("ISBN10").alias("International_Standard_Book_Number"),
                col("asin").alias("Amazon_Standard_Identification_Number"),
                col("title").alias("Title"),
                col("brand").alias("Author"),
                col("availability").alias("Availability"),
                col("currency").alias("Currency"),
                col("discount").alias("Discount"),
                col("final_price").alias("Final_Price"),
                col("initial_price").alias("Initial_Price"),
                col("rating_value").alias("Rating"),
                col("reviews_count").alias("Reviews_Count"),
                col("seller_name").alias("Seller_Name"),
                col("item_weight").alias("Item_Weight"))
        .filter("ISBN10 is not null")).dropDuplicates()
    return df_all_books

In [0]:
@dlt.table(
    name=f"gold.{schema}.best_books"
    )
def best_books():
    df = dlt.readStream(f"{silver_schema}.{param_source_name}")
    df_best_books = (
        df.select(
            col("title").alias("Title"),
            col("brand").alias("Author"),
            col("rating_value").alias("Rating"),
            col("reviews_count").alias("Reviews_Count"),
            col("final_price").alias("Final_Price")
        )
        .filter(col("rating_value") >= 4.9)
        .dropDuplicates()
    )
    return df_best_books